In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
collection = ee.ImageCollection('MODIS/006/MYD13A1').select('EVI')

In [ ]:
reference = collection.filterDate('2001-01-01', '2010-12-31')\
    .sort('system:time_start', False)
mean = reference.mean()

In [ ]:
def function(image):
    return image.subtract(mean).set('system:time_start',
                                    image.get('system:time_start'))


series = collection.filterDate('2011-01-01', '2014-12-31').map(function)

In [ ]:
Map.setCenter(-100.811, 40.2, 5)
Map.addLayer(series.sum(),
             {'min': -60000, 'max': 60000,
              'palette': ['FF0000', '000000', '00FF00']},
             'EVI anomaly')

In [ ]:
time0 = reference.first().get('system:time_start')
first = ee.List([
    ee.Image(0).set('system:time_start', time0).select([0], ['EVI'])
])

In [ ]:
def accumulate(image, List):
    previous = ee.Image(ee.List(List).get(-1))
    added = image.add(previous).set('system:time_start',
                                    image.get('system:time_start'))
    return ee.List(List).add(added)

In [ ]:
cumulative = ee.ImageCollection(ee.List(series.iterate(accumulate, first)))